# Gradient Boost

In [1]:
# Imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# to enable local imports
module_path = os.path.abspath('../code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from machine_learning_load_data import loadOnlineEEGdata

/home/nvidia/masterthesis/thesis_eeg/code


In [3]:
# Load some online EEG Data
eegData, freqData, entropyData = loadOnlineEEGdata(dirPath='../../EEG_Data/eeg_data_online', splitData=True)
eegX_train, eegy_train, eegX_test, eegy_test = eegData
freqX_train, freqy_train, freqX_test, freqy_test = freqData
X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy = entropyData

# reshape
freqX_train = freqX_train.reshape(freqX_train.shape[0], freqX_train.shape[2])
freqX_test = freqX_test.reshape(freqX_test.shape[0], freqX_test.shape[2])

X_train_entropy = X_train_entropy.reshape(X_train_entropy.shape[0], X_train_entropy.shape[2])
X_test_entropy = X_test_entropy.reshape(X_test_entropy.shape[0], X_test_entropy.shape[2])

Loading Online EEG Data from ../../EEG_Data/eeg_data_online ...
EEG Data Shape:
(5024, 512, 40) (5024,) (2154, 512, 40) (2154,)
Freq Data Shape:
(1008, 1, 1200) (1008,) (432, 1, 1200) (432,)
Entropy Data Shape:
(5024, 1, 200) (5024,) (2154, 1, 200) (2154,)


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score)

model = GradientBoostingClassifier()


# Create a parameter grid - here you specifiy which combinations you want to test
param_grid = [
        {'learning_rate' : [0.001, 0.01, 0.1],
         'loss' : ['deviance', 'exponential'],
         'n_estimators' : [1000], # 1000 seems good 
         #'subsample' : [1.0],
         #'criterion' : ['friedman_mse', 'mse', 'mae'],
         #'min_samples_split' : [2],
         #'min_samples_leaf' : [1],
         #'max_depth' : [3]
    }
]

# Create randomized search 5-fold cross validation and 100 iterations
grid_search = GridSearchCV(model, param_grid, cv=5,
                            scoring=f1_scorer,
                            return_train_score=True,
                            n_jobs=-1)

# fit it with the data
result = grid_search.fit(X_train_entropy, y_train_entropy)

In [6]:
print("Best Params: {}".format(grid_search.best_params_))
print("Best Estimator: {}".format(grid_search.best_estimator_))

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

Best Params: {'loss': 'exponential', 'n_estimators': 1000, 'learning_rate': 0.1}
Best Estimator: GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='exponential', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
0.42863237872500404 {'loss': 'deviance', 'n_estimators': 1000, 'learning_rate': 0.001}
0.4283832877465391 {'loss': 'exponential', 'n_estimators': 1000, 'learning_rate': 0.001}
0.5333009558304342 {'loss'